<a href="https://colab.research.google.com/github/avbendre/Generative_AI/blob/main/Data_Extractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q gradio openai pypdf tiktoken langchain PyPDF2 pandas chromadb -q

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "[HIDDEN_KEY]"

In [ ]:
import PyPDF2
import gradio as gr
import openai
from io import BytesIO
from langchain import OpenAI, PromptTemplate
import langchain
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser
import requests
from bs4 import BeautifulSoup
import pandas as pd
from langchain.prompts import ChatPromptTemplate

In [ ]:


#We use here the response schemas as we need to get the output in structured format

Name_schema = ResponseSchema(name="Name",
                             description="What is the name of the person under Obituary")
Birthday_schema = ResponseSchema(name="Birthday",
                                      description="What is the birthday of the Person")
Husband_name_schema = ResponseSchema(name="Husband_Name",
                                    description="What is the name of the husband")
Children_schema=ResponseSchema(name="Children",
                                    description="What is the number of children")
Marraige_date_schema=ResponseSchema(name="Marriage_Date",
                                    description="What is the Date of marriage")
grandchildren_schema=ResponseSchema(name="grandchildren",
                                    description="What is the number of grandchildren in number")
greatgrandchildren_schema=ResponseSchema(name="greatgrandchildren",
                                    description="What is the number of greatgrandchildren in number")


response_schemas = [Name_schema, 
                    Birthday_schema,
                    Husband_name_schema,Children_schema,Marraige_date_schema,grandchildren_schema
                    ,greatgrandchildren_schema]


#We take the template to be such that we get the output in form of json
review_template_2 = """\
For the following text, extract the following information:

Name: What is the name of the person under Obituary?\

Husband_Name: What is the name of the husband?\

Children: What is the number of children?\

Marraige date: When is the marriage date?\

grandchildren: What is the number of grandchildren in number?\

greatgrandchildren: What is the number of greatgrandchildren in number?\

Format the output as JSON with the following keys:
Name
Husband_Name
Children
Marraige date
grandchildren
greatgrandchildren.

and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""
#Using Beautiful soup Function to parse the text content from the website
def extract_text(url):
    response = requests.get(url)
    html_content = response.text

    soup = BeautifulSoup(html_content, 'html.parser')
    text = soup.get_text()

    return text


def main():
    website_url = input("Enter the website URL: ")
    extracted_text = extract_text(website_url)
    output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
    format_instructions = output_parser.get_format_instructions()
    prompt = ChatPromptTemplate.from_template(template=review_template_2)

    messages = prompt.format_messages(text=extracted_text, 
                                format_instructions=format_instructions)
   
    chat = ChatOpenAI(temperature=0.0)
    response = chat(messages)
    #Json is the output but it is in string format, so we need to use parser to convert it to desired dict
    output_dict = output_parser.parse(response.content)
    #Converting the dict into an dataframe
    df = pd.DataFrame([output_dict])
    print(df.head())


if __name__ == "__main__":
    main()


Enter the website URL: https://www.chappellfuneralhome.com/obituary/Patrice-McCarty#obituary
                              Name        Birthday  Husband_Name Children  \
0  Patrice Ann (Williams)  McCarty  April 21, 1960  Rick McCarty        2   

       Marriage_Date grandchildren greatgrandchildren  
0  December 21, 1994             4                  1  
